In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
import numpy as np
from metric import metrics_stat

%load_ext autoreload
%autoreload 2
from indices import MainDataset
from neighbors import Neighborhoods
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
from metric import metrics_stat

In [2]:
train_dataset_index = MainDataset("dataset/train.csv")
test_dataset_index = MainDataset("dataset/test.csv", need_index = False)

/home/ivan/projects/raif/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
neighborhoods = Neighborhoods(train_dataset_index.index)

In [4]:
train_dataset = pd.read_csv("dataset/train.csv")
train_dataset=train_dataset[train_dataset['price_type'] == 1]
test_dataset = pd.read_csv("dataset/test.csv")
train_dataset["is_train"] = 1
test_dataset["is_train"] = 0
dataset = pd.concat([train_dataset,test_dataset])

/home/ivan/projects/raif/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
dataset["neighbor_dist"]=-999
dataset["neighbor_total_price"]=-999
dataset["neighbor_square_price"]=-999
for d in [test_dataset_index, train_dataset_index]:
    for i, o in enumerate(d.all_objects):
        if o.row["price_type"] != 1:
            continue
        neighbor = neighborhoods.get_haversine_closest(o, 12)
        n = neighbor[0]
        dataset.loc[dataset["id"] == o.row["id"], "neighbor_dist"] = n[1]
        dataset.loc[dataset["id"] == o.row["id"], "neighbor_total_price"] = n[0].row["per_square_meter_price"]* n[0].row["total_square"]
        dataset.loc[dataset["id"] == o.row["id"], "neighbor_square_price"] = n[0].row["per_square_meter_price"]
        
        dataset.loc[dataset["id"] == o.row["id"], "neighbor10_dist"] = neighbor[10][1]

In [6]:
floor_mapping = {"Цоколь": -1, "цокольный":-1, "Подвал": -1,"подвал": -1,"цоколь": -1,
                 "-1.0": -1,"0.0": -1,"-2.0": -1, "Техническое подполье":-1,
                   
                 "1 этаж": 1,"1.0": 1,  "1-й":1, "фактически на уровне 1 этажа":1, "1, антресоль":1,
                 "2.0": 2, "2-й":2,
                 "3.0": 3, "3 этаж":3,
                   "4.0":4, "4 этаж":4,
                  "5.0": 5,
                 "6.0": 6,
                 "7.0": 7,
                 "8.0": 8,
                 "9.0": 9,
                 "10.0": 10,
                 
                 "1,2":1,"1, 2":1,"1,2,3,4":1, "1, 2, 3":1, "1,2,3,4,5":1, "1-й, 2-й":1, "1, 2 этаж":1,
                 "подвал, 1, 2, 3":1, "подвал, 1":1,  "1,2,подвал":1,
                 "1,2,3, антресоль, технический этаж":1, "1 (по док-м цоколь)":1,
                 "цокольный, 1,2":1,"1-7":1, "3, Мансарда (4 эт)":3, "1,2,подвал":1,
                 
                 "1, подвал":0,  "1, цоколь":0, 
                  "цоколь, 1":0, "подвал, 1 и 4 этаж":0,
                 
                  "подвал, 3. 4 этаж":3,
                 "29":100, "36":100
                }

floor_count_mapping = {
    "1, подвал":2,
     "1,2":2,
    "1, 2":2,
    "1,2,3,4":2,
    "цоколь, 1, 2,3,4,5,6":7,
    "1, 2, 3":3,
    "цоколь, 1":2,
    "подвал , 1":2,
    
    "1, цоколь":2,
    "Цоколь, 1,2(мансарда)":3,
    "подвал,1,2,3,4,5":6,
    "1-3 этажи":3,
    "1 этаж, подвал":2,
    "1,2,3,4, подвал":5,
    "подвал,1,2,3":4,
    "подвал , 1":2,
    "1,2,3 этаж, подвал":4,
    "1,2,3,4,5":5,
    "2, 3, 4, тех.этаж":4,
    "технический этаж,5,6":3,
    " 1-2, подвальный":3,
    "1, 2, 3, мансардный":4,
    "1,2":2,
    "подвал, 1 и 4 этаж":3,
    "4, 5":2,
    "1 + подвал (без отделки)":2,
    "подвал, 1,2,3":4,
    "подвал, 1-4 этаж":5,
    
    "1,2,антресоль":2,
    " 1, 2, Антресоль":2,
    "подва, 1.2 этаж":3,
    "подвал, 1-7, техэтаж":8,
    "1,2,3":3,
    "3,4":2,
    "1-й, подвал":2,
    "1-й, 2-й":2,
    "1,2,подвал":3,
    "1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)":3,
    "1, подвал, антресоль":2,
    "1.2":2,
    "1-3":3,
    "1,2,подвал":3,
    "подвал , 1":2,
    "подвал, 3. 4 этаж":3,
    "1, 2 этаж":2,
    "подвал, 1, 2, 3":3,
    "подвал, 1":2,
    "1,2,подвал":3,
    "1,2,3, антресоль, технический этаж":4,
    "цокольный, 1,2":3,
    "1-7":8,
    "3, Мансарда (4 эт)":2,
    "1,2,подвал":3,
  
    
}

has_basement = [
    "подвал","цоколь","1, подвал","цокольный","Техническое подполье","цоколь, 1","1, цоколь","1 этаж, подвал","подвал , 1","Цоколь, 1,2(мансарда)",
     "1,2,3 этаж, подвал","цоколь, 1, 2,3,4,5,6","подвал, 1","1-й, подвал","1 (по док-м цоколь)","подвал, 2","подвал, 1 и 4 этаж","2, 3, 4, тех.этаж",
     "1,2,3,4, подвал","подвал,1,2,3,4,5","1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)","технический этаж,5,6","подвал,1,2,3","подва, 1.2 этаж",
     "подвал, цоколь, 1 этаж","подвал, 1, 2","подвал, 1-4 этаж","1, подвал, антресоль","1 + подвал (без отделки)"," 1-2, подвальный","подвал, 3. 4 этаж",
     "подвал, 1,2,3","тех.этаж (6)","цокольный, 1,2","1,2,3, антресоль, технический этаж","Цоколь","1,2,подвал","подвал, 1, 2, 3","1 (Цокольный этаж)","подвал, 1-3","подвал, 1-7, техэтаж"
]

dataset["has_basement"]=0
dataset.loc[  dataset["floor"].isna(), "has_basement"] = -999

for k in has_basement:
    dataset.loc[  dataset["floor"] == k, "has_basement"] = 1
    
    
dataset["floor_count"]=1
dataset.loc[  dataset["floor"].isna(), "floor_count"] = -999
for k,v in floor_count_mapping.items():
    dataset.loc[  dataset["floor"] == k, "floor_count"] = v

for k,v in floor_mapping.items():
    dataset.loc[  dataset["floor"] == k, "floor"] = v
dataset["floor"]=dataset["floor"].fillna(-999)
dataset["floor"]=dataset.floor.astype("str")

In [7]:
pd.set_option("display.max_rows", None) 
dataset[dataset["floor_count"]<2].floor.value_counts()


-999                         3765
1                            2187
-1                            443
2                             375
3                             193
4                             122
5                              86
6                              44
9                              16
7                              15
10                             12
8                              10
13                              6
100                             5
14                              4
подвал , 1                      3
5(мансарда)                     3
мансарда                        3
12.0                            3
2,3                             3
11.0                            2
15.0                            2
17.0                            2
5, мансарда                     2
подвал,1                        2
18                              1
тех.этаж (6)                    1
1,2                             1
5 этаж                          1
13.0          

In [8]:
street_pop = dataset.street.value_counts().to_dict()
dataset["street_pop"] = -999 
dataset["street_pop"] =dataset[~dataset.street.isna()].street.apply(lambda x:street_pop[x])

In [9]:
city_pop = dataset.city.value_counts().to_dict()
dataset["city_pop"] = -999 
dataset["city_pop"] =dataset[~dataset.city.isna()].city.apply(lambda x:city_pop[x])

region_pop = dataset.region.value_counts().to_dict()
dataset["region_pop"] = -999 
dataset["region_pop"] =dataset[~dataset.region.isna()].region.apply(lambda x:region_pop[x])

In [10]:
dataset["floor_square"] =dataset['total_square']/dataset['floor_count']
dataset.loc[dataset['floor_count'] < 1, "floor_square"] = -999


dataset["expect_price"] =dataset['neighbor_total_price']/dataset['total_square']



In [11]:
dense_features = [  'lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
       'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
       'osm_historic_points_in_0.005', 'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01', 
       'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square', "floor_count", "has_basement","street_pop",'city_pop','region_pop',
        "neighbor_dist", "neighbor_total_price", "neighbor_square_price", "neighbor10_dist","floor_square",
                  "expect_price"
       ]

cat_features = ['city', 'osm_city_nearest_name', 'region', 'realty_type', 'street', 'floor']

all_features = []
all_features.extend(dense_features)
all_features.extend(cat_features)

In [12]:
dataset = dataset.fillna(-999)

In [13]:
X = dataset[dataset.is_train==1][all_features]
Y = dataset[dataset.is_train==1].per_square_meter_price
X_test = dataset[dataset.is_train==0][all_features]



In [37]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=46)

In [38]:
Y_train_spb = Y_train[X_train.city == "Санкт-Петербург"]
X_train_spb = X_train[X_train.city == "Санкт-Петербург"]

Y_val_spb = Y_val[X_val.city == "Санкт-Петербург"]
X_val_spb = X_val[X_val.city == "Санкт-Петербург"]

In [39]:
full_train_data = Pool(data=X,
                  cat_features=cat_features,
                  label=np.log(np.array(Y)))


train_data_spb = Pool(data=X_train_spb,
                  cat_features=cat_features,
                  label=np.log(np.array(Y_train_spb)))

val_data_spb= Pool(data=X_val_spb,
                cat_features=cat_features,
                  label=np.log(np.array(Y_val_spb)))

train_data = Pool(data=X_train,
                  cat_features=cat_features,
                  label=np.log(np.array(Y_train)))

val_data = Pool(data=X_val,
                cat_features=cat_features,
                  label=np.log(np.array(Y_val)))

test_data = Pool(data=X_test,
                  cat_features=cat_features)

In [49]:
cat_model = CatBoostRegressor(
#     l2_leaf_reg=6,
#     bagging_temperature=1.5,
#     random_strength=2,
    learning_rate=0.01,
    iterations=10000,
    eval_metric="RMSE",
)
cat_model.fit(train_data,eval_set=val_data, use_best_model=False)

0:	learn: 0.5859449	test: 0.5692339	best: 0.5692339 (0)	total: 5.42ms	remaining: 54.2s
1:	learn: 0.5831601	test: 0.5663206	best: 0.5663206 (1)	total: 13.4ms	remaining: 1m 7s
2:	learn: 0.5804954	test: 0.5634828	best: 0.5634828 (2)	total: 18.5ms	remaining: 1m 1s
3:	learn: 0.5777740	test: 0.5606845	best: 0.5606845 (3)	total: 23.9ms	remaining: 59.8s
4:	learn: 0.5751195	test: 0.5577954	best: 0.5577954 (4)	total: 34.2ms	remaining: 1m 8s
5:	learn: 0.5723210	test: 0.5549505	best: 0.5549505 (5)	total: 40.6ms	remaining: 1m 7s
6:	learn: 0.5696101	test: 0.5521484	best: 0.5521484 (6)	total: 46.6ms	remaining: 1m 6s
7:	learn: 0.5668747	test: 0.5492846	best: 0.5492846 (7)	total: 51.7ms	remaining: 1m 4s
8:	learn: 0.5643208	test: 0.5466921	best: 0.5466921 (8)	total: 57.2ms	remaining: 1m 3s
9:	learn: 0.5619183	test: 0.5441598	best: 0.5441598 (9)	total: 61.7ms	remaining: 1m 1s
10:	learn: 0.5594308	test: 0.5415675	best: 0.5415675 (10)	total: 67.6ms	remaining: 1m 1s
11:	learn: 0.5567709	test: 0.5388129	best

103:	learn: 0.4210066	test: 0.3998154	best: 0.3998154 (103)	total: 603ms	remaining: 57.4s
104:	learn: 0.4201665	test: 0.3989322	best: 0.3989322 (104)	total: 613ms	remaining: 57.7s
105:	learn: 0.4193573	test: 0.3982086	best: 0.3982086 (105)	total: 618ms	remaining: 57.7s
106:	learn: 0.4186134	test: 0.3974893	best: 0.3974893 (106)	total: 623ms	remaining: 57.6s
107:	learn: 0.4178413	test: 0.3967100	best: 0.3967100 (107)	total: 630ms	remaining: 57.7s
108:	learn: 0.4170698	test: 0.3959449	best: 0.3959449 (108)	total: 636ms	remaining: 57.7s
109:	learn: 0.4162355	test: 0.3952379	best: 0.3952379 (109)	total: 643ms	remaining: 57.8s
110:	learn: 0.4154768	test: 0.3945151	best: 0.3945151 (110)	total: 648ms	remaining: 57.7s
111:	learn: 0.4148594	test: 0.3938047	best: 0.3938047 (111)	total: 653ms	remaining: 57.6s
112:	learn: 0.4141744	test: 0.3930762	best: 0.3930762 (112)	total: 659ms	remaining: 57.7s
113:	learn: 0.4134618	test: 0.3923634	best: 0.3923634 (113)	total: 665ms	remaining: 57.7s
114:	learn

198:	learn: 0.3731934	test: 0.3541142	best: 0.3541142 (198)	total: 1.21s	remaining: 59.4s
199:	learn: 0.3729259	test: 0.3538826	best: 0.3538826 (199)	total: 1.21s	remaining: 59.4s
200:	learn: 0.3725747	test: 0.3535964	best: 0.3535964 (200)	total: 1.22s	remaining: 59.5s
201:	learn: 0.3723370	test: 0.3533761	best: 0.3533761 (201)	total: 1.23s	remaining: 59.5s
202:	learn: 0.3720348	test: 0.3531240	best: 0.3531240 (202)	total: 1.23s	remaining: 59.6s
203:	learn: 0.3716852	test: 0.3527669	best: 0.3527669 (203)	total: 1.24s	remaining: 59.5s
204:	learn: 0.3714004	test: 0.3525244	best: 0.3525244 (204)	total: 1.24s	remaining: 59.5s
205:	learn: 0.3710530	test: 0.3521912	best: 0.3521912 (205)	total: 1.25s	remaining: 59.4s
206:	learn: 0.3708013	test: 0.3519339	best: 0.3519339 (206)	total: 1.25s	remaining: 59.4s
207:	learn: 0.3704555	test: 0.3516112	best: 0.3516112 (207)	total: 1.26s	remaining: 59.3s
208:	learn: 0.3701390	test: 0.3513383	best: 0.3513383 (208)	total: 1.26s	remaining: 59.3s
209:	learn

304:	learn: 0.3497756	test: 0.3345239	best: 0.3345239 (304)	total: 1.81s	remaining: 57.5s
305:	learn: 0.3495766	test: 0.3344135	best: 0.3344135 (305)	total: 1.82s	remaining: 57.6s
306:	learn: 0.3494661	test: 0.3343164	best: 0.3343164 (306)	total: 1.82s	remaining: 57.5s
307:	learn: 0.3493000	test: 0.3341644	best: 0.3341644 (307)	total: 1.83s	remaining: 57.6s
308:	learn: 0.3491247	test: 0.3340982	best: 0.3340982 (308)	total: 1.83s	remaining: 57.6s
309:	learn: 0.3490059	test: 0.3340344	best: 0.3340344 (309)	total: 1.84s	remaining: 57.5s
310:	learn: 0.3488701	test: 0.3339526	best: 0.3339526 (310)	total: 1.84s	remaining: 57.5s
311:	learn: 0.3487276	test: 0.3339560	best: 0.3339526 (310)	total: 1.85s	remaining: 57.5s
312:	learn: 0.3485788	test: 0.3338565	best: 0.3338565 (312)	total: 1.86s	remaining: 57.4s
313:	learn: 0.3484509	test: 0.3337399	best: 0.3337399 (313)	total: 1.86s	remaining: 57.4s
314:	learn: 0.3482952	test: 0.3336822	best: 0.3336822 (314)	total: 1.87s	remaining: 57.4s
315:	learn

413:	learn: 0.3356241	test: 0.3255857	best: 0.3255857 (413)	total: 2.41s	remaining: 55.8s
414:	learn: 0.3354536	test: 0.3255114	best: 0.3255114 (414)	total: 2.42s	remaining: 55.9s
415:	learn: 0.3353178	test: 0.3254551	best: 0.3254551 (415)	total: 2.42s	remaining: 55.9s
416:	learn: 0.3351999	test: 0.3253906	best: 0.3253906 (416)	total: 2.43s	remaining: 55.8s
417:	learn: 0.3351371	test: 0.3253826	best: 0.3253826 (417)	total: 2.44s	remaining: 55.9s
418:	learn: 0.3350290	test: 0.3253108	best: 0.3253108 (418)	total: 2.44s	remaining: 55.8s
419:	learn: 0.3349430	test: 0.3253033	best: 0.3253033 (419)	total: 2.45s	remaining: 55.8s
420:	learn: 0.3348019	test: 0.3252420	best: 0.3252420 (420)	total: 2.45s	remaining: 55.8s
421:	learn: 0.3346917	test: 0.3251973	best: 0.3251973 (421)	total: 2.46s	remaining: 55.7s
422:	learn: 0.3346119	test: 0.3251656	best: 0.3251656 (422)	total: 2.46s	remaining: 55.7s
423:	learn: 0.3345043	test: 0.3250908	best: 0.3250908 (423)	total: 2.46s	remaining: 55.7s
424:	learn

...

9516:	learn: 0.1240409	test: 0.3415723	best: 0.3169679 (926)	total: 56.6s	remaining: 2.87s
9517:	learn: 0.1240403	test: 0.3415738	best: 0.3169679 (926)	total: 56.6s	remaining: 2.87s
9518:	learn: 0.1240365	test: 0.3415731	best: 0.3169679 (926)	total: 56.6s	remaining: 2.86s
9519:	learn: 0.1240302	test: 0.3415779	best: 0.3169679 (926)	total: 56.6s	remaining: 2.85s
9520:	learn: 0.1240240	test: 0.3415744	best: 0.3169679 (926)	total: 56.6s	remaining: 2.85s
9521:	learn: 0.1240203	test: 0.3415732	best: 0.3169679 (926)	total: 56.6s	remaining: 2.84s
9522:	learn: 0.1240071	test: 0.3415749	best: 0.3169679 (926)	total: 56.6s	remaining: 2.83s
9523:	learn: 0.1239993	test: 0.3415730	best: 0.3169679 (926)	total: 56.6s	remaining: 2.83s
9524:	learn: 0.1239952	test: 0.3415706	best: 0.3169679 (926)	total: 56.6s	remaining: 2.82s
9525:	learn: 0.1239841	test: 0.3415724	best: 0.3169679 (926)	total: 56.6s	remaining: 2.82s
9526:	learn: 0.1239734	test: 0.3415849	best: 0.3169679 (926)	total: 56.6s	remaining: 2.81s

9628:	learn: 0.1230695	test: 0.3416006	best: 0.3169679 (926)	total: 57.2s	remaining: 2.2s
9629:	learn: 0.1230466	test: 0.3415847	best: 0.3169679 (926)	total: 57.2s	remaining: 2.2s
9630:	learn: 0.1230441	test: 0.3415861	best: 0.3169679 (926)	total: 57.2s	remaining: 2.19s
9631:	learn: 0.1230434	test: 0.3415860	best: 0.3169679 (926)	total: 57.2s	remaining: 2.19s
9632:	learn: 0.1230356	test: 0.3415826	best: 0.3169679 (926)	total: 57.2s	remaining: 2.18s
9633:	learn: 0.1230196	test: 0.3415812	best: 0.3169679 (926)	total: 57.2s	remaining: 2.17s
9634:	learn: 0.1230192	test: 0.3415812	best: 0.3169679 (926)	total: 57.2s	remaining: 2.17s
9635:	learn: 0.1230100	test: 0.3415799	best: 0.3169679 (926)	total: 57.2s	remaining: 2.16s
9636:	learn: 0.1230034	test: 0.3415821	best: 0.3169679 (926)	total: 57.2s	remaining: 2.15s
9637:	learn: 0.1229905	test: 0.3415891	best: 0.3169679 (926)	total: 57.2s	remaining: 2.15s
9638:	learn: 0.1229877	test: 0.3415876	best: 0.3169679 (926)	total: 57.2s	remaining: 2.14s
9

9739:	learn: 0.1221105	test: 0.3417009	best: 0.3169679 (926)	total: 57.8s	remaining: 1.54s
9740:	learn: 0.1220988	test: 0.3417125	best: 0.3169679 (926)	total: 57.8s	remaining: 1.54s
9741:	learn: 0.1220850	test: 0.3417176	best: 0.3169679 (926)	total: 57.8s	remaining: 1.53s
9742:	learn: 0.1220660	test: 0.3417154	best: 0.3169679 (926)	total: 57.8s	remaining: 1.52s
9743:	learn: 0.1220554	test: 0.3417266	best: 0.3169679 (926)	total: 57.8s	remaining: 1.52s
9744:	learn: 0.1220552	test: 0.3417264	best: 0.3169679 (926)	total: 57.8s	remaining: 1.51s
9745:	learn: 0.1220360	test: 0.3417256	best: 0.3169679 (926)	total: 57.8s	remaining: 1.51s
9746:	learn: 0.1220279	test: 0.3417209	best: 0.3169679 (926)	total: 57.8s	remaining: 1.5s
9747:	learn: 0.1220206	test: 0.3417261	best: 0.3169679 (926)	total: 57.8s	remaining: 1.49s
9748:	learn: 0.1220098	test: 0.3417298	best: 0.3169679 (926)	total: 57.8s	remaining: 1.49s
9749:	learn: 0.1220080	test: 0.3417308	best: 0.3169679 (926)	total: 57.8s	remaining: 1.48s


9850:	learn: 0.1211124	test: 0.3417679	best: 0.3169679 (926)	total: 58.4s	remaining: 883ms
9851:	learn: 0.1211022	test: 0.3417669	best: 0.3169679 (926)	total: 58.4s	remaining: 877ms
9852:	learn: 0.1210959	test: 0.3417689	best: 0.3169679 (926)	total: 58.4s	remaining: 871ms
9853:	learn: 0.1210931	test: 0.3417685	best: 0.3169679 (926)	total: 58.4s	remaining: 865ms
9854:	learn: 0.1210766	test: 0.3417647	best: 0.3169679 (926)	total: 58.4s	remaining: 859ms
9855:	learn: 0.1210688	test: 0.3417621	best: 0.3169679 (926)	total: 58.4s	remaining: 853ms
9856:	learn: 0.1210525	test: 0.3417637	best: 0.3169679 (926)	total: 58.4s	remaining: 847ms
9857:	learn: 0.1210523	test: 0.3417630	best: 0.3169679 (926)	total: 58.4s	remaining: 842ms
9858:	learn: 0.1210407	test: 0.3417646	best: 0.3169679 (926)	total: 58.4s	remaining: 836ms
9859:	learn: 0.1210268	test: 0.3417696	best: 0.3169679 (926)	total: 58.4s	remaining: 830ms
9860:	learn: 0.1210213	test: 0.3417700	best: 0.3169679 (926)	total: 58.4s	remaining: 824ms

9959:	learn: 0.1200985	test: 0.3419388	best: 0.3169679 (926)	total: 59s	remaining: 237ms
9960:	learn: 0.1200884	test: 0.3419384	best: 0.3169679 (926)	total: 59s	remaining: 231ms
9961:	learn: 0.1200823	test: 0.3419348	best: 0.3169679 (926)	total: 59s	remaining: 225ms
9962:	learn: 0.1200753	test: 0.3419353	best: 0.3169679 (926)	total: 59s	remaining: 219ms
9963:	learn: 0.1200660	test: 0.3419297	best: 0.3169679 (926)	total: 59s	remaining: 213ms
9964:	learn: 0.1200486	test: 0.3419268	best: 0.3169679 (926)	total: 59s	remaining: 207ms
9965:	learn: 0.1200421	test: 0.3419235	best: 0.3169679 (926)	total: 59s	remaining: 201ms
9966:	learn: 0.1200190	test: 0.3419329	best: 0.3169679 (926)	total: 59s	remaining: 195ms
9967:	learn: 0.1200157	test: 0.3419280	best: 0.3169679 (926)	total: 59s	remaining: 189ms
9968:	learn: 0.1199975	test: 0.3419205	best: 0.3169679 (926)	total: 59s	remaining: 184ms
9969:	learn: 0.1199807	test: 0.3419288	best: 0.3169679 (926)	total: 59s	remaining: 178ms
9970:	learn: 0.119965

In [41]:
cat_preds = cat_model.predict(X_val, ntree_end=8000)
metrics_stat(np.array(Y_val), np.exp(np.array(cat_preds))/1.15)

{'mape': 0.5448041005627919,
 'mdape': 0.4004732547261428,
 'rmse': 44844.56841454855,
 'r2': 0.5993084376290987,
 'raif_metric': 4.520078517485215}

In [42]:
cat_preds = cat_model.predict(X_train_spb, ntree_end=8000)
metrics_stat(np.array(Y_train_spb), np.exp(np.array(cat_preds))/1.15)

{'mape': 0.13206023923376364,
 'mdape': 0.128816991781533,
 'rmse': 54085.961951966936,
 'r2': 0.9174332340659468,
 'raif_metric': 0.052634868740745244}

In [58]:
cat_preds = cat_model.predict(X_val_spb, ntree_end=10000)
metrics_stat(np.array(Y_val_spb), np.exp(np.array(cat_preds))/1.11)

{'mape': 0.3255652087367841,
 'mdape': 0.20854663567331527,
 'rmse': 123058.16233425603,
 'r2': 0.46170016343758047,
 'raif_metric': 2.614018038977448}

In [63]:
test_predicts = []
test_predicts_spb = []
test_data = Pool(data=X_test,
                  cat_features=cat_features)

X_test_spb = X_test[X_test.city == "Санкт-Петербург"]
test_data_spb = Pool(data=X_test_spb,
                  cat_features=cat_features)

for seed in range(10,20):
    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=seed)
    train_data = Pool(data=X_train,
                  cat_features=cat_features,
                  label=np.log(np.array(Y_train)))
    
    Y_train_spb = Y_train[X_train.city == "Санкт-Петербург"]
    X_train_spb = X_train[X_train.city == "Санкт-Петербург"]
    
    train_data_spb = Pool(data=X_train_spb,
                  cat_features=cat_features,
                  label=np.log(np.array(Y_train_spb)))
    
    cat_model = CatBoostRegressor(
        learning_rate=0.01,
        iterations=100,
        eval_metric="RMSE",
    )
    cat_model.fit(train_data)
    test_predicts.append(cat_model.predict(test_data))
    
    cat_model = CatBoostRegressor(
        learning_rate=0.01,
        iterations=100,
        eval_metric="RMSE",
    )
    cat_model.fit(train_data)
    test_predicts_spb.append(cat_model.predict(test_data_spb))

0:	learn: 0.5877386	total: 1.83ms	remaining: 181ms
1:	learn: 0.5849296	total: 6.19ms	remaining: 304ms
2:	learn: 0.5821129	total: 9.86ms	remaining: 319ms
3:	learn: 0.5792509	total: 13.7ms	remaining: 329ms
4:	learn: 0.5767025	total: 19ms	remaining: 361ms
5:	learn: 0.5741472	total: 22.8ms	remaining: 357ms
6:	learn: 0.5715816	total: 27.2ms	remaining: 361ms
7:	learn: 0.5689076	total: 31.6ms	remaining: 363ms
8:	learn: 0.5662761	total: 37ms	remaining: 374ms
9:	learn: 0.5638614	total: 41.3ms	remaining: 372ms
10:	learn: 0.5614168	total: 45.1ms	remaining: 365ms
11:	learn: 0.5587283	total: 49ms	remaining: 359ms
12:	learn: 0.5562979	total: 52.9ms	remaining: 354ms
13:	learn: 0.5540272	total: 56.5ms	remaining: 347ms
14:	learn: 0.5516384	total: 60.1ms	remaining: 341ms
15:	learn: 0.5494447	total: 64.7ms	remaining: 340ms
16:	learn: 0.5470103	total: 68.4ms	remaining: 334ms
17:	learn: 0.5447190	total: 72.1ms	remaining: 329ms
18:	learn: 0.5423240	total: 75.8ms	remaining: 323ms
19:	learn: 0.5400081	total: 

89:	learn: 0.4359142	total: 358ms	remaining: 39.8ms
90:	learn: 0.4349688	total: 364ms	remaining: 36ms
91:	learn: 0.4341311	total: 369ms	remaining: 32.1ms
92:	learn: 0.4332040	total: 374ms	remaining: 28.1ms
93:	learn: 0.4322950	total: 380ms	remaining: 24.2ms
94:	learn: 0.4313925	total: 385ms	remaining: 20.2ms
95:	learn: 0.4304761	total: 388ms	remaining: 16.2ms
96:	learn: 0.4296007	total: 392ms	remaining: 12.1ms
97:	learn: 0.4287025	total: 396ms	remaining: 8.08ms
98:	learn: 0.4278157	total: 400ms	remaining: 4.04ms
99:	learn: 0.4270019	total: 409ms	remaining: 0us
0:	learn: 0.5821523	total: 2.33ms	remaining: 231ms
1:	learn: 0.5794328	total: 9.21ms	remaining: 451ms
2:	learn: 0.5767689	total: 15.6ms	remaining: 505ms
3:	learn: 0.5740712	total: 21.4ms	remaining: 513ms
4:	learn: 0.5715463	total: 28.2ms	remaining: 536ms
5:	learn: 0.5689822	total: 34.1ms	remaining: 535ms
6:	learn: 0.5665079	total: 39.7ms	remaining: 527ms
7:	learn: 0.5639161	total: 45.3ms	remaining: 521ms
8:	learn: 0.5611981	total

92:	learn: 0.4270497	total: 405ms	remaining: 30.5ms
93:	learn: 0.4261181	total: 410ms	remaining: 26.2ms
94:	learn: 0.4253131	total: 415ms	remaining: 21.8ms
95:	learn: 0.4243897	total: 418ms	remaining: 17.4ms
96:	learn: 0.4235436	total: 422ms	remaining: 13ms
97:	learn: 0.4226603	total: 425ms	remaining: 8.68ms
98:	learn: 0.4218431	total: 429ms	remaining: 4.34ms
99:	learn: 0.4209963	total: 434ms	remaining: 0us
0:	learn: 0.5852175	total: 1.53ms	remaining: 152ms
1:	learn: 0.5824359	total: 6.02ms	remaining: 295ms
2:	learn: 0.5796481	total: 9.81ms	remaining: 317ms
3:	learn: 0.5767297	total: 14.2ms	remaining: 340ms
4:	learn: 0.5740976	total: 18.5ms	remaining: 352ms
5:	learn: 0.5714412	total: 23ms	remaining: 360ms
6:	learn: 0.5688127	total: 27.3ms	remaining: 362ms
7:	learn: 0.5660178	total: 32.1ms	remaining: 370ms
8:	learn: 0.5634808	total: 36.9ms	remaining: 374ms
9:	learn: 0.5610215	total: 40.7ms	remaining: 367ms
10:	learn: 0.5583936	total: 47.2ms	remaining: 382ms
11:	learn: 0.5559087	total: 5

61:	learn: 0.4645084	total: 258ms	remaining: 158ms
62:	learn: 0.4632331	total: 265ms	remaining: 155ms
63:	learn: 0.4619851	total: 269ms	remaining: 151ms
64:	learn: 0.4606398	total: 273ms	remaining: 147ms
65:	learn: 0.4593525	total: 278ms	remaining: 143ms
66:	learn: 0.4581825	total: 282ms	remaining: 139ms
67:	learn: 0.4569435	total: 286ms	remaining: 135ms
68:	learn: 0.4557384	total: 290ms	remaining: 130ms
69:	learn: 0.4545509	total: 294ms	remaining: 126ms
70:	learn: 0.4533168	total: 299ms	remaining: 122ms
71:	learn: 0.4519783	total: 304ms	remaining: 118ms
72:	learn: 0.4507714	total: 308ms	remaining: 114ms
73:	learn: 0.4495989	total: 313ms	remaining: 110ms
74:	learn: 0.4484816	total: 317ms	remaining: 106ms
75:	learn: 0.4473468	total: 320ms	remaining: 101ms
76:	learn: 0.4461891	total: 326ms	remaining: 97.3ms
77:	learn: 0.4449871	total: 331ms	remaining: 93.2ms
78:	learn: 0.4440191	total: 335ms	remaining: 89.1ms
79:	learn: 0.4428499	total: 340ms	remaining: 85ms
80:	learn: 0.4417280	total: 3

37:	learn: 0.5049258	total: 149ms	remaining: 243ms
38:	learn: 0.5030109	total: 153ms	remaining: 239ms
39:	learn: 0.5011799	total: 157ms	remaining: 235ms
40:	learn: 0.4993590	total: 161ms	remaining: 231ms
41:	learn: 0.4975927	total: 165ms	remaining: 228ms
42:	learn: 0.4959368	total: 171ms	remaining: 227ms
43:	learn: 0.4943289	total: 176ms	remaining: 224ms
44:	learn: 0.4926690	total: 180ms	remaining: 220ms
45:	learn: 0.4910482	total: 190ms	remaining: 223ms
46:	learn: 0.4893553	total: 195ms	remaining: 220ms
47:	learn: 0.4877333	total: 200ms	remaining: 216ms
48:	learn: 0.4860936	total: 204ms	remaining: 212ms
49:	learn: 0.4845897	total: 207ms	remaining: 207ms
50:	learn: 0.4830072	total: 211ms	remaining: 203ms
51:	learn: 0.4814057	total: 216ms	remaining: 199ms
52:	learn: 0.4799471	total: 219ms	remaining: 194ms
53:	learn: 0.4785313	total: 223ms	remaining: 190ms
54:	learn: 0.4769897	total: 227ms	remaining: 186ms
55:	learn: 0.4756156	total: 231ms	remaining: 182ms
56:	learn: 0.4742743	total: 235

...

50:	learn: 0.4821150	total: 201ms	remaining: 193ms
51:	learn: 0.4804669	total: 208ms	remaining: 192ms
52:	learn: 0.4788264	total: 212ms	remaining: 188ms
53:	learn: 0.4772783	total: 215ms	remaining: 184ms
54:	learn: 0.4759428	total: 219ms	remaining: 179ms
55:	learn: 0.4746239	total: 223ms	remaining: 175ms
56:	learn: 0.4731225	total: 231ms	remaining: 174ms
57:	learn: 0.4718003	total: 234ms	remaining: 170ms
58:	learn: 0.4702897	total: 238ms	remaining: 165ms
59:	learn: 0.4688819	total: 243ms	remaining: 162ms
60:	learn: 0.4674679	total: 247ms	remaining: 158ms
61:	learn: 0.4661138	total: 251ms	remaining: 154ms
62:	learn: 0.4648434	total: 254ms	remaining: 149ms
63:	learn: 0.4635323	total: 258ms	remaining: 145ms
64:	learn: 0.4622995	total: 262ms	remaining: 141ms
65:	learn: 0.4610860	total: 265ms	remaining: 137ms
66:	learn: 0.4599552	total: 269ms	remaining: 133ms
67:	learn: 0.4585989	total: 273ms	remaining: 128ms
68:	learn: 0.4572093	total: 277ms	remaining: 124ms
69:	learn: 0.4561796	total: 280

17:	learn: 0.5422378	total: 72.2ms	remaining: 329ms
18:	learn: 0.5397835	total: 77.1ms	remaining: 329ms
19:	learn: 0.5374228	total: 82ms	remaining: 328ms
20:	learn: 0.5351129	total: 86.1ms	remaining: 324ms
21:	learn: 0.5326373	total: 89.7ms	remaining: 318ms
22:	learn: 0.5304036	total: 94.2ms	remaining: 315ms
23:	learn: 0.5281992	total: 97.8ms	remaining: 310ms
24:	learn: 0.5261320	total: 102ms	remaining: 305ms
25:	learn: 0.5240748	total: 105ms	remaining: 300ms
26:	learn: 0.5219186	total: 110ms	remaining: 297ms
27:	learn: 0.5198925	total: 113ms	remaining: 290ms
28:	learn: 0.5178050	total: 117ms	remaining: 288ms
29:	learn: 0.5157423	total: 121ms	remaining: 283ms
30:	learn: 0.5138462	total: 125ms	remaining: 278ms
31:	learn: 0.5120114	total: 129ms	remaining: 273ms
32:	learn: 0.5100391	total: 133ms	remaining: 269ms
33:	learn: 0.5082335	total: 136ms	remaining: 264ms
34:	learn: 0.5062670	total: 140ms	remaining: 260ms
35:	learn: 0.5044689	total: 143ms	remaining: 255ms
36:	learn: 0.5026856	total

0:	learn: 0.5820012	total: 1.49ms	remaining: 147ms
1:	learn: 0.5793696	total: 5.36ms	remaining: 263ms
2:	learn: 0.5766915	total: 9.67ms	remaining: 313ms
3:	learn: 0.5738743	total: 14.6ms	remaining: 351ms
4:	learn: 0.5714002	total: 18.9ms	remaining: 359ms
5:	learn: 0.5687787	total: 23.6ms	remaining: 369ms
6:	learn: 0.5662635	total: 27.7ms	remaining: 368ms
7:	learn: 0.5634664	total: 31.7ms	remaining: 364ms
8:	learn: 0.5608068	total: 35.3ms	remaining: 357ms
9:	learn: 0.5581793	total: 39.6ms	remaining: 356ms
10:	learn: 0.5558347	total: 43.8ms	remaining: 354ms
11:	learn: 0.5533524	total: 47.9ms	remaining: 351ms
12:	learn: 0.5508885	total: 52ms	remaining: 348ms
13:	learn: 0.5487368	total: 55.6ms	remaining: 342ms
14:	learn: 0.5463994	total: 59.9ms	remaining: 340ms
15:	learn: 0.5441283	total: 64.4ms	remaining: 338ms
16:	learn: 0.5418307	total: 67.9ms	remaining: 331ms
17:	learn: 0.5397551	total: 71.7ms	remaining: 327ms
18:	learn: 0.5373365	total: 75.3ms	remaining: 321ms
19:	learn: 0.5351610	tot

78:	learn: 0.4431085	total: 336ms	remaining: 89.4ms
79:	learn: 0.4420059	total: 344ms	remaining: 86ms
80:	learn: 0.4409602	total: 349ms	remaining: 81.8ms
81:	learn: 0.4398607	total: 352ms	remaining: 77.4ms
82:	learn: 0.4389189	total: 357ms	remaining: 73.1ms
83:	learn: 0.4379348	total: 361ms	remaining: 68.7ms
84:	learn: 0.4369122	total: 364ms	remaining: 64.3ms
85:	learn: 0.4360098	total: 368ms	remaining: 59.9ms
86:	learn: 0.4351112	total: 373ms	remaining: 55.7ms
87:	learn: 0.4341098	total: 377ms	remaining: 51.4ms
88:	learn: 0.4331940	total: 381ms	remaining: 47.1ms
89:	learn: 0.4322310	total: 384ms	remaining: 42.7ms
90:	learn: 0.4313450	total: 388ms	remaining: 38.4ms
91:	learn: 0.4305083	total: 392ms	remaining: 34.1ms
92:	learn: 0.4296494	total: 396ms	remaining: 29.8ms
93:	learn: 0.4287366	total: 399ms	remaining: 25.5ms
94:	learn: 0.4278514	total: 403ms	remaining: 21.2ms
95:	learn: 0.4270806	total: 407ms	remaining: 16.9ms
96:	learn: 0.4262780	total: 410ms	remaining: 12.7ms
97:	learn: 0.4

48:	learn: 0.4772451	total: 206ms	remaining: 214ms
49:	learn: 0.4757059	total: 212ms	remaining: 212ms
50:	learn: 0.4741394	total: 216ms	remaining: 208ms
51:	learn: 0.4725457	total: 221ms	remaining: 204ms
52:	learn: 0.4709940	total: 224ms	remaining: 199ms
53:	learn: 0.4694853	total: 230ms	remaining: 196ms
54:	learn: 0.4681275	total: 234ms	remaining: 191ms
55:	learn: 0.4667980	total: 238ms	remaining: 187ms
56:	learn: 0.4653314	total: 241ms	remaining: 182ms
57:	learn: 0.4638449	total: 246ms	remaining: 178ms
58:	learn: 0.4624720	total: 250ms	remaining: 174ms
59:	learn: 0.4610230	total: 254ms	remaining: 169ms
60:	learn: 0.4597149	total: 257ms	remaining: 165ms
61:	learn: 0.4584369	total: 261ms	remaining: 160ms
62:	learn: 0.4570923	total: 265ms	remaining: 156ms
63:	learn: 0.4557802	total: 269ms	remaining: 151ms
64:	learn: 0.4545078	total: 272ms	remaining: 147ms
65:	learn: 0.4532140	total: 276ms	remaining: 142ms
66:	learn: 0.4518920	total: 280ms	remaining: 138ms
67:	learn: 0.4505669	total: 284

In [68]:
max(preds_spb)

123613.5716347487

In [66]:
preds_spb=np.exp(np.mean(test_predicts_spb, axis=0))/1.1


10      COL_289763
41      COL_291738
54      COL_292654
61      COL_293588
100     COL_296521
123     COL_298459
129     COL_298593
135     COL_298968
137     COL_299221
154     COL_299678
161     COL_300108
163     COL_300144
165     COL_300305
184     COL_301574
190     COL_301757
224     COL_303405
237     COL_304255
260     COL_305114
272     COL_305726
341     COL_310641
343     COL_310656
355     COL_311328
361     COL_311704
382     COL_312997
392     COL_313559
436     COL_315673
467     COL_316897
483     COL_318698
491     COL_319542
513     COL_320942
518     COL_321128
558     COL_323355
562     COL_323683
565     COL_324197
568     COL_324455
587     COL_325545
603     COL_325942
604     COL_325998
704     COL_333473
707     COL_333545
718     COL_334091
721     COL_334288
741     COL_336462
751     COL_336963
816     COL_340794
818     COL_340898
823     COL_341069
867     COL_343169
889     COL_344969
892     COL_345062
894     COL_345117
895     COL_345119
898     COL_

In [89]:
preds=np.exp(np.mean(test_predicts, axis=0))/1.15

result = pd.DataFrame({"id": dataset[dataset.is_train==0].id,
             "per_square_meter_price": preds
             })
result.loc[result.id.isin(dataset[(dataset.is_train==0) & (dataset.city == "Санкт-Петербург")].id), "per_square_meter_price"]=preds_spb
result.to_csv("spb.csv", index=False)

In [80]:
dataset[(dataset.is_train==0) & (dataset.city == "Санкт-Петербург")].id==result.id

ValueError: Can only compare identically-labeled Series objects

In [54]:
{'mape': 0.24468737251480635,
 'mdape': 0.20646907117596422,
 'rmse': 41430.27721278688,
 'r2': 0.6579999431184261,
 'raif_metric': 1.3071931406851722}

floor_count 	0.416733
floor_count 	0.323210

SyntaxError: invalid syntax (<ipython-input-54-ab52440a5c77>, line 7)

In [68]:
pd.set_option("display.max_rows", None) 
cat_model.get_feature_importance(data=val_data,prettified=True)

Feature Id  Importances
0                                region     6.757698
1                          floor_square     5.657248
2                          total_square     5.513768
3                                  city     5.085360
4                 neighbor_square_price     3.898065
5                                 floor     3.056891
6                 osm_city_nearest_name     2.909590
7                  neighbor_total_price     2.455980
8             osm_crossing_closest_dist     2.434517
9                                   lng     2.167133
10              osm_subway_closest_dist     2.148671
11      osm_transport_stop_closest_dist     2.091675
12                          realty_type     2.053883
13         osm_catering_points_in_0.005     1.948595
14          osm_catering_points_in_0.01     1.860649
15          osm_city_nearest_population     1.821132
16                osm_city_closest_dist     1.730796
17                               street     1.615182
18          reform_mean_floor_count_500     1.575625
19        osm_catering_points_in_0.0075     1.555463
20                             city_pop     1.550302
21          osm_crossing_points_in_0.01     1.509032
22                        neighbor_dist     1.503852
23         osm_amenity_points_in_0.0075     1.455817
24         reform_house_population_1000     1.399025
25         reform_mean_floor_count_1000     1.347500
26          osm_amenity_points_in_0.001     1.342870
27        reform_mean_year_building_500     1.238777
28                      neighbor10_dist     1.212723
29         osm_catering_points_in_0.001     1.208595
30                           region_pop     1.187971
31          reform_count_of_houses_1000     1.154834
32          reform_house_population_500     1.141866
33       reform_mean_year_building_1000     1.129707
34          osm_train_stop_closest_dist     1.118489
35            osm_hotels_points_in_0.01     1.085609
36           osm_amenity_points_in_0.01     1.061824
37          osm_amenity_points_in_0.005     1.033670
38                                  lat     1.016021
39         osm_crossing_points_in_0.005     0.982155
40                           street_pop     0.894854
41        osm_crossing_points_in_0.0075     0.846274
42    osm_transport_stop_points_in_0.01     0.760450
43  osm_transport_stop_points_in_0.0075     0.721878
44           osm_offices_points_in_0.01     0.628621
45         osm_offices_points_in_0.0075     0.618503
46           osm_leisure_points_in_0.01     0.609429
47           osm_hotels_points_in_0.005     0.609288
48            osm_shops_points_in_0.005     0.602681
49           reform_count_of_houses_500     0.599188
50        osm_healthcare_points_in_0.01     0.584309
51   osm_transport_stop_points_in_0.005     0.574716
52        osm_historic_points_in_0.0075     0.574123
53          osm_historic_points_in_0.01     0.566677
54           osm_shops_points_in_0.0075     0.542136
55         osm_finance_points_in_0.0075     0.536913
56          osm_finance_points_in_0.005     0.534661
57           osm_finance_points_in_0.01     0.511575
58         osm_historic_points_in_0.005     0.435585
59          osm_leisure_points_in_0.005     0.424957
60         osm_crossing_points_in_0.001     0.422165
61           osm_culture_points_in_0.01     0.415923
62          osm_hotels_points_in_0.0075     0.365978
63             osm_shops_points_in_0.01     0.343672
64          osm_finance_points_in_0.001     0.338028
65            osm_shops_points_in_0.001     0.328510
66         osm_culture_points_in_0.0075     0.326053
67          osm_offices_points_in_0.005     0.312870
68      osm_healthcare_points_in_0.0075     0.294062
69          osm_culture_points_in_0.005     0.278740
70         osm_leisure_points_in_0.0075     0.242152
71       osm_healthcare_points_in_0.005     0.221569
72          osm_culture_points_in_0.001     0.221195
73          osm_offices_points_in_0.001     0.211728
74        osm_building_points_in_0.0075     0.130474
7

In [20]:
cat_model = CatBoostRegressor(
    l2_leaf_reg=6,
    bagging_temperature=1.5,
#     random_strength=2,
    learning_rate=0.014,
    iterations=8000,
    eval_metric="RMSE",
)
cat_model.fit(full_train_data)

0:	learn: 0.5834842	total: 14.9ms	remaining: 1m 59s
1:	learn: 0.5797745	total: 24.1ms	remaining: 1m 36s
2:	learn: 0.5760892	total: 29.6ms	remaining: 1m 18s
3:	learn: 0.5725653	total: 38.7ms	remaining: 1m 17s
4:	learn: 0.5687566	total: 43.5ms	remaining: 1m 9s
5:	learn: 0.5651086	total: 48.3ms	remaining: 1m 4s
6:	learn: 0.5616078	total: 53.2ms	remaining: 1m
7:	learn: 0.5579628	total: 59.2ms	remaining: 59.2s
8:	learn: 0.5546193	total: 64.1ms	remaining: 56.9s
9:	learn: 0.5511886	total: 68.7ms	remaining: 54.9s
10:	learn: 0.5480799	total: 73.4ms	remaining: 53.3s
11:	learn: 0.5449597	total: 78ms	remaining: 51.9s
12:	learn: 0.5418540	total: 82.7ms	remaining: 50.8s
13:	learn: 0.5385844	total: 87.8ms	remaining: 50.1s
14:	learn: 0.5354826	total: 92.1ms	remaining: 49s
15:	learn: 0.5325813	total: 96.7ms	remaining: 48.2s
16:	learn: 0.5297651	total: 102ms	remaining: 48s
17:	learn: 0.5269663	total: 107ms	remaining: 47.3s
18:	learn: 0.5242328	total: 111ms	remaining: 46.8s
19:	learn: 0.5215875	total: 11

188:	learn: 0.3617630	total: 1.01s	remaining: 41.7s
189:	learn: 0.3614996	total: 1.01s	remaining: 41.7s
190:	learn: 0.3612621	total: 1.02s	remaining: 41.8s
191:	learn: 0.3610018	total: 1.03s	remaining: 41.8s
192:	learn: 0.3607465	total: 1.03s	remaining: 41.8s
193:	learn: 0.3604558	total: 1.04s	remaining: 41.8s
194:	learn: 0.3602775	total: 1.04s	remaining: 41.8s
195:	learn: 0.3600586	total: 1.05s	remaining: 41.7s
196:	learn: 0.3598375	total: 1.05s	remaining: 41.7s
197:	learn: 0.3595921	total: 1.06s	remaining: 41.7s
198:	learn: 0.3592983	total: 1.06s	remaining: 41.8s
199:	learn: 0.3590746	total: 1.07s	remaining: 41.8s
200:	learn: 0.3588203	total: 1.08s	remaining: 41.8s
201:	learn: 0.3586544	total: 1.08s	remaining: 41.8s
202:	learn: 0.3583944	total: 1.09s	remaining: 41.8s
203:	learn: 0.3582067	total: 1.09s	remaining: 41.8s
204:	learn: 0.3579762	total: 1.1s	remaining: 41.8s
205:	learn: 0.3578176	total: 1.1s	remaining: 41.7s
206:	learn: 0.3575880	total: 1.11s	remaining: 41.9s
207:	learn: 0.

383:	learn: 0.3330248	total: 2s	remaining: 39.8s
384:	learn: 0.3329459	total: 2.01s	remaining: 39.8s
385:	learn: 0.3328767	total: 2.02s	remaining: 39.8s
386:	learn: 0.3327891	total: 2.02s	remaining: 39.8s
387:	learn: 0.3327093	total: 2.03s	remaining: 39.8s
388:	learn: 0.3326428	total: 2.04s	remaining: 39.9s
389:	learn: 0.3325398	total: 2.04s	remaining: 39.9s
390:	learn: 0.3324604	total: 2.05s	remaining: 39.9s
391:	learn: 0.3323787	total: 2.05s	remaining: 39.9s
392:	learn: 0.3323140	total: 2.06s	remaining: 39.8s
393:	learn: 0.3322526	total: 2.06s	remaining: 39.8s
394:	learn: 0.3321874	total: 2.07s	remaining: 39.8s
395:	learn: 0.3320839	total: 2.07s	remaining: 39.8s
396:	learn: 0.3319515	total: 2.08s	remaining: 39.8s
397:	learn: 0.3318673	total: 2.08s	remaining: 39.8s
398:	learn: 0.3318181	total: 2.09s	remaining: 39.8s
399:	learn: 0.3317517	total: 2.09s	remaining: 39.8s
400:	learn: 0.3316377	total: 2.1s	remaining: 39.8s
401:	learn: 0.3315648	total: 2.1s	remaining: 39.7s
402:	learn: 0.331

568:	learn: 0.3191277	total: 3s	remaining: 39.2s
569:	learn: 0.3190947	total: 3.01s	remaining: 39.2s
570:	learn: 0.3190464	total: 3.02s	remaining: 39.2s
571:	learn: 0.3189771	total: 3.02s	remaining: 39.2s
572:	learn: 0.3188917	total: 3.02s	remaining: 39.2s
573:	learn: 0.3188413	total: 3.03s	remaining: 39.2s
574:	learn: 0.3187655	total: 3.04s	remaining: 39.2s
575:	learn: 0.3187133	total: 3.04s	remaining: 39.2s
576:	learn: 0.3186554	total: 3.05s	remaining: 39.2s
577:	learn: 0.3186063	total: 3.05s	remaining: 39.2s
578:	learn: 0.3185442	total: 3.06s	remaining: 39.2s
579:	learn: 0.3185030	total: 3.06s	remaining: 39.2s
580:	learn: 0.3184429	total: 3.07s	remaining: 39.2s
581:	learn: 0.3183715	total: 3.07s	remaining: 39.1s
582:	learn: 0.3183426	total: 3.08s	remaining: 39.1s
583:	learn: 0.3183233	total: 3.08s	remaining: 39.1s
584:	learn: 0.3182558	total: 3.08s	remaining: 39.1s
585:	learn: 0.3182158	total: 3.09s	remaining: 39.1s
586:	learn: 0.3181850	total: 3.1s	remaining: 39.1s
587:	learn: 0.31

751:	learn: 0.3078641	total: 4s	remaining: 38.6s
752:	learn: 0.3078085	total: 4.01s	remaining: 38.6s
753:	learn: 0.3077162	total: 4.01s	remaining: 38.6s
754:	learn: 0.3076571	total: 4.02s	remaining: 38.6s
755:	learn: 0.3075821	total: 4.02s	remaining: 38.6s
756:	learn: 0.3074742	total: 4.03s	remaining: 38.6s
757:	learn: 0.3074405	total: 4.04s	remaining: 38.6s
758:	learn: 0.3073606	total: 4.04s	remaining: 38.5s
759:	learn: 0.3072876	total: 4.04s	remaining: 38.5s
760:	learn: 0.3072029	total: 4.05s	remaining: 38.5s
761:	learn: 0.3071161	total: 4.05s	remaining: 38.5s
762:	learn: 0.3070692	total: 4.06s	remaining: 38.5s
763:	learn: 0.3069701	total: 4.07s	remaining: 38.5s
764:	learn: 0.3068805	total: 4.07s	remaining: 38.5s
765:	learn: 0.3068061	total: 4.08s	remaining: 38.5s
766:	learn: 0.3067599	total: 4.08s	remaining: 38.5s
767:	learn: 0.3066478	total: 4.09s	remaining: 38.5s
768:	learn: 0.3065459	total: 4.09s	remaining: 38.5s
769:	learn: 0.3065049	total: 4.1s	remaining: 38.5s
770:	learn: 0.30

...

7121:	learn: 0.1633750	total: 38.2s	remaining: 4.7s
7122:	learn: 0.1633636	total: 38.2s	remaining: 4.7s
7123:	learn: 0.1633609	total: 38.2s	remaining: 4.69s
7124:	learn: 0.1633532	total: 38.2s	remaining: 4.69s
7125:	learn: 0.1633266	total: 38.2s	remaining: 4.68s
7126:	learn: 0.1633077	total: 38.2s	remaining: 4.68s
7127:	learn: 0.1632976	total: 38.2s	remaining: 4.67s
7128:	learn: 0.1632652	total: 38.2s	remaining: 4.67s
7129:	learn: 0.1632639	total: 38.2s	remaining: 4.66s
7130:	learn: 0.1632625	total: 38.2s	remaining: 4.66s
7131:	learn: 0.1632411	total: 38.2s	remaining: 4.65s
7132:	learn: 0.1632302	total: 38.2s	remaining: 4.64s
7133:	learn: 0.1632205	total: 38.2s	remaining: 4.64s
7134:	learn: 0.1632021	total: 38.2s	remaining: 4.63s
7135:	learn: 0.1631784	total: 38.2s	remaining: 4.63s
7136:	learn: 0.1631547	total: 38.2s	remaining: 4.62s
7137:	learn: 0.1631392	total: 38.2s	remaining: 4.62s
7138:	learn: 0.1631346	total: 38.3s	remaining: 4.61s
7139:	learn: 0.1631211	total: 38.3s	remaining: 4

7313:	learn: 0.1610176	total: 39.2s	remaining: 3.67s
7314:	learn: 0.1610116	total: 39.2s	remaining: 3.67s
7315:	learn: 0.1610054	total: 39.2s	remaining: 3.66s
7316:	learn: 0.1609751	total: 39.2s	remaining: 3.66s
7317:	learn: 0.1609571	total: 39.2s	remaining: 3.65s
7318:	learn: 0.1609475	total: 39.2s	remaining: 3.65s
7319:	learn: 0.1609284	total: 39.2s	remaining: 3.64s
7320:	learn: 0.1609094	total: 39.2s	remaining: 3.63s
7321:	learn: 0.1609021	total: 39.2s	remaining: 3.63s
7322:	learn: 0.1609017	total: 39.2s	remaining: 3.62s
7323:	learn: 0.1608792	total: 39.2s	remaining: 3.62s
7324:	learn: 0.1608619	total: 39.2s	remaining: 3.61s
7325:	learn: 0.1608537	total: 39.2s	remaining: 3.61s
7326:	learn: 0.1608419	total: 39.2s	remaining: 3.6s
7327:	learn: 0.1608235	total: 39.2s	remaining: 3.6s
7328:	learn: 0.1608064	total: 39.2s	remaining: 3.59s
7329:	learn: 0.1607873	total: 39.2s	remaining: 3.59s
7330:	learn: 0.1607592	total: 39.2s	remaining: 3.58s
7331:	learn: 0.1607470	total: 39.3s	remaining: 3

7492:	learn: 0.1587566	total: 40.2s	remaining: 2.72s
7493:	learn: 0.1587462	total: 40.2s	remaining: 2.71s
7494:	learn: 0.1587234	total: 40.2s	remaining: 2.71s
7495:	learn: 0.1587083	total: 40.2s	remaining: 2.7s
7496:	learn: 0.1587014	total: 40.2s	remaining: 2.7s
7497:	learn: 0.1587009	total: 40.2s	remaining: 2.69s
7498:	learn: 0.1586630	total: 40.2s	remaining: 2.69s
7499:	learn: 0.1586622	total: 40.2s	remaining: 2.68s
7500:	learn: 0.1586380	total: 40.2s	remaining: 2.67s
7501:	learn: 0.1586338	total: 40.2s	remaining: 2.67s
7502:	learn: 0.1586286	total: 40.2s	remaining: 2.66s
7503:	learn: 0.1586182	total: 40.2s	remaining: 2.66s
7504:	learn: 0.1586179	total: 40.2s	remaining: 2.65s
7505:	learn: 0.1586133	total: 40.2s	remaining: 2.65s
7506:	learn: 0.1586076	total: 40.3s	remaining: 2.64s
7507:	learn: 0.1586049	total: 40.3s	remaining: 2.64s
7508:	learn: 0.1585877	total: 40.3s	remaining: 2.63s
7509:	learn: 0.1585749	total: 40.3s	remaining: 2.63s
7510:	learn: 0.1585643	total: 40.3s	remaining: 2

7670:	learn: 0.1567347	total: 41.2s	remaining: 1.76s
7671:	learn: 0.1566997	total: 41.2s	remaining: 1.76s
7672:	learn: 0.1566746	total: 41.2s	remaining: 1.75s
7673:	learn: 0.1566592	total: 41.2s	remaining: 1.75s
7674:	learn: 0.1566498	total: 41.2s	remaining: 1.74s
7675:	learn: 0.1566373	total: 41.2s	remaining: 1.74s
7676:	learn: 0.1566217	total: 41.2s	remaining: 1.73s
7677:	learn: 0.1565966	total: 41.2s	remaining: 1.73s
7678:	learn: 0.1565813	total: 41.2s	remaining: 1.72s
7679:	learn: 0.1565667	total: 41.2s	remaining: 1.72s
7680:	learn: 0.1565533	total: 41.2s	remaining: 1.71s
7681:	learn: 0.1565298	total: 41.2s	remaining: 1.71s
7682:	learn: 0.1565209	total: 41.3s	remaining: 1.7s
7683:	learn: 0.1565206	total: 41.3s	remaining: 1.7s
7684:	learn: 0.1565034	total: 41.3s	remaining: 1.69s
7685:	learn: 0.1564970	total: 41.3s	remaining: 1.69s
7686:	learn: 0.1564824	total: 41.3s	remaining: 1.68s
7687:	learn: 0.1564804	total: 41.3s	remaining: 1.68s
7688:	learn: 0.1564697	total: 41.3s	remaining: 1

7851:	learn: 0.1544394	total: 42.2s	remaining: 795ms
7852:	learn: 0.1544253	total: 42.2s	remaining: 789ms
7853:	learn: 0.1544156	total: 42.2s	remaining: 784ms
7854:	learn: 0.1544044	total: 42.2s	remaining: 779ms
7855:	learn: 0.1543997	total: 42.2s	remaining: 773ms
7856:	learn: 0.1543977	total: 42.2s	remaining: 768ms
7857:	learn: 0.1543793	total: 42.2s	remaining: 763ms
7858:	learn: 0.1543609	total: 42.2s	remaining: 757ms
7859:	learn: 0.1543515	total: 42.2s	remaining: 752ms
7860:	learn: 0.1543305	total: 42.2s	remaining: 747ms
7861:	learn: 0.1543196	total: 42.2s	remaining: 741ms
7862:	learn: 0.1543063	total: 42.2s	remaining: 736ms
7863:	learn: 0.1542893	total: 42.2s	remaining: 730ms
7864:	learn: 0.1542814	total: 42.2s	remaining: 725ms
7865:	learn: 0.1542807	total: 42.2s	remaining: 720ms
7866:	learn: 0.1542730	total: 42.3s	remaining: 714ms
7867:	learn: 0.1542727	total: 42.3s	remaining: 709ms
7868:	learn: 0.1542579	total: 42.3s	remaining: 704ms
7869:	learn: 0.1542408	total: 42.3s	remaining:

In [74]:
pd.set_option("display.max_rows", None) 
cat_model.get_feature_importance(data=val_data,prettified=True)

Feature Id  Importances
0                          total_square    13.104249
1                                  city     7.041973
2                                region     6.750129
3                 osm_city_nearest_name     4.405499
4                                 floor     2.772086
5             osm_crossing_closest_dist     2.686790
6       osm_transport_stop_closest_dist     2.490756
7                 osm_city_closest_dist     2.387722
8                           realty_type     2.369187
9               osm_subway_closest_dist     2.285619
10          osm_catering_points_in_0.01     2.252175
11                                  lng     2.185542
12                               street     2.125652
13          osm_crossing_points_in_0.01     1.995914
14        osm_catering_points_in_0.0075     1.938094
15         osm_catering_points_in_0.005     1.775639
16          reform_mean_floor_count_500     1.773333
17        reform_mean_year_building_500     1.742989
18         reform_mean_floor_count_1000     1.704602
19         reform_house_population_1000     1.609776
20    osm_transport_stop_points_in_0.01     1.547595
21          osm_city_nearest_population     1.535510
22          osm_train_stop_closest_dist     1.381885
23                                  lat     1.368306
24          osm_amenity_points_in_0.001     1.366048
25           osm_amenity_points_in_0.01     1.337233
26          reform_count_of_houses_1000     1.335571
27       reform_mean_year_building_1000     1.285420
28          reform_house_population_500     1.244006
29         osm_crossing_points_in_0.005     1.215944
30            osm_hotels_points_in_0.01     1.147565
31          osm_amenity_points_in_0.005     1.059238
32          osm_hotels_points_in_0.0075     1.031725
33         osm_catering_points_in_0.001     1.004854
34        osm_crossing_points_in_0.0075     0.924530
35           osm_shops_points_in_0.0075     0.867002
36            osm_shops_points_in_0.005     0.862452
37         osm_finance_points_in_0.0075     0.744639
38           reform_count_of_houses_500     0.721641
39             osm_shops_points_in_0.01     0.684509
40  osm_transport_stop_points_in_0.0075     0.676765
41         osm_amenity_points_in_0.0075     0.673300
42           osm_finance_points_in_0.01     0.646500
43         osm_crossing_points_in_0.001     0.603481
44          osm_historic_points_in_0.01     0.589685
45   osm_transport_stop_points_in_0.005     0.563164
46        osm_historic_points_in_0.0075     0.549200
47        osm_healthcare_points_in_0.01     0.523082
48           osm_offices_points_in_0.01     0.515541
49           osm_leisure_points_in_0.01     0.502828
50           osm_culture_points_in_0.01     0.484879
51          osm_finance_points_in_0.001     0.480474
52      osm_healthcare_points_in_0.0075     0.449846
53          osm_finance_points_in_0.005     0.449415
54          osm_offices_points_in_0.005     0.425012
55           osm_hotels_points_in_0.005     0.407171
56         osm_historic_points_in_0.005     0.397606
57          osm_leisure_points_in_0.005     0.393229
58         osm_offices_points_in_0.0075     0.362769
59         osm_leisure_points_in_0.0075     0.306791
60          osm_culture_points_in_0.005     0.289241
61            osm_shops_points_in_0.001     0.287181
62       osm_healthcare_points_in_0.005     0.265231
63         osm_building_points_in_0.005     0.208561
64         osm_culture_points_in_0.0075     0.208140
65        osm_building_points_in_0.0075     0.165664
66          osm_offices_points_in_0.001     0.157511
67      osm_train_stop_points_in_0.0075     0.116748
68          osm_building_points_in_0.01     0.104877
69          osm_culture_points_in_0.001     0.078767
70        osm_train_stop_points_in_0.01     0.030738
71       osm_train_stop_points_in_0.005     0.020616
72         osm_building_points_in_0.001     0.002588

In [22]:
price0_preds =  np.exp(cat_model.predict(test_data))/1.15

price0_preds

array([38197.60589473, 36979.58845732, 34577.19255868, ...,
       35446.11048361, 39899.19260806, 41631.90872404])

In [24]:
pd.DataFrame({"id": dataset[dataset.is_train==0].id,
             "per_square_meter_price": price0_preds
             }).to_csv("log3.csv", index=False)